In [2]:
"""
FFIEC Call Report Feature Filtering Script (Step 3 Track 2 - v0.2)
===================================================================
Purpose: Take individual per-bank CSVs from Step 2 Track 2 and filter
         each bank down to a clean set of numeric features suitable
         for per-bank anomaly detection / ML.

Approach: Same data-driven filtering as Step 3 (null %, variance,
          unique ratio, duplicates), but applied independently to
          each bank. This means each bank may end up with a DIFFERENT
          set of surviving features, which is appropriate for per-bank
          ML models.

OUTPUT FORMAT (v0.2 change): The output CSVs are TRANSPOSED so that:
  - ROWS    = feature names (MDRM codes / field names)
  - COLUMNS = quarters (e.g., '03/31/2000', '06/30/2000', ...)
  This orientation makes it easy to read each feature's time series
  across quarters horizontally.

Input:  per_bank/ffiec_<bank_name>.csv  (from Step 2 Track 2)
Output: per_bank_features/ffiec_<bank_name>_features.csv  (transposed, one per bank)
        per_bank_features/filtering_report_<bank_name>.txt
        per_bank_features/feature_comparison.txt  (cross-bank summary)

Author: Wake Forest MSBA Practicum Team 4
Date: February 2026
"""

import pandas as pd
import numpy as np
from pathlib import Path
import re
from datetime import datetime


# =============================================================================
# CONFIGURATION
# =============================================================================

INPUT_DIR = Path("per_bank")
OUTPUT_DIR = Path("per_bank_features")

# Filtering thresholds (same as original Step 3)
NULL_THRESHOLD = 0.0
VARIANCE_THRESHOLD = 0.001
UNIQUE_RATIO_THRESHOLD = 0.01

# Metadata columns to keep but not treat as features
METADATA_COLUMNS = [
    'IDRSSD',
    'quarter',
    'FDIC Certificate Number',
    'OCC Charter Number',
    'OTS Docket Number',
    'Primary ABA Routing Number',
    'Financial Institution Name',
    'Financial Institution Address',
    'Financial Institution City',
    'Financial Institution State',
    'Financial Institution Zip Code',
]

# Bank name mapping (for display)
BANKS = {
    '480228': 'Bank of America',
    '852218': 'JPMorgan Chase Bank',
    '476810': 'Citibank',
    '451965': 'Wells Fargo Bank',
    '2182786': 'Goldman Sachs Bank USA',
    '1456501': 'Morgan Stanley Bank',
}


# =============================================================================
# HELPER FUNCTIONS (identical to Step 3 v0.3)
# =============================================================================

def identify_column_types(df):
    """Categorize columns into metadata, numeric, and non-numeric."""
    all_cols = set(df.columns)
    metadata = set(METADATA_COLUMNS) & all_cols
    
    numeric_cols = []
    non_numeric_cols = []
    
    for col in all_cols - metadata:
        if col.startswith('Unnamed:'):
            non_numeric_cols.append(col)
            continue
        if pd.api.types.is_numeric_dtype(df[col]):
            numeric_cols.append(col)
        else:
            try:
                converted = pd.to_numeric(df[col], errors='coerce')
                if converted.notna().mean() > 0.5:
                    numeric_cols.append(col)
                else:
                    non_numeric_cols.append(col)
            except:
                non_numeric_cols.append(col)
    
    return {
        'metadata': list(metadata),
        'numeric': numeric_cols,
        'non_numeric': non_numeric_cols
    }


def filter_by_null_percentage(df, columns, threshold):
    """Remove columns with too many null values."""
    null_pct = df[columns].isnull().mean()
    kept = null_pct[null_pct <= threshold].index.tolist()
    removed = null_pct[null_pct > threshold]
    removed_with_reasons = {
        col: f"Null percentage: {pct:.1%}"
        for col, pct in removed.items()
    }
    return kept, removed_with_reasons


def filter_by_variance(df, columns, threshold):
    """Remove columns with near-zero variance."""
    kept = []
    removed_with_reasons = {}
    
    for col in columns:
        series = pd.to_numeric(df[col], errors='coerce')
        if series.notna().sum() == 0:
            removed_with_reasons[col] = "All values null after numeric conversion"
            continue
        std = series.std()
        if std == 0 or pd.isna(std):
            removed_with_reasons[col] = f"Zero variance (constant value: {series.dropna().iloc[0] if series.notna().any() else 'N/A'})"
        else:
            normalized = (series - series.mean()) / std
            var = normalized.var()
            if var < threshold:
                removed_with_reasons[col] = f"Near-zero variance: {var:.6f}"
            else:
                kept.append(col)
    
    return kept, removed_with_reasons


def filter_by_unique_ratio(df, columns, threshold):
    """Remove columns with very few unique values relative to dataset size."""
    kept = []
    removed_with_reasons = {}
    n_rows = len(df)
    
    for col in columns:
        series = pd.to_numeric(df[col], errors='coerce').dropna()
        if len(series) == 0:
            removed_with_reasons[col] = "No valid numeric values"
            continue
        n_unique = series.nunique()
        ratio = n_unique / n_rows
        if n_unique <= 3:
            removed_with_reasons[col] = f"Only {n_unique} unique values (likely categorical/flag)"
        elif ratio < threshold:
            removed_with_reasons[col] = f"Low unique ratio: {n_unique} unique / {n_rows} rows = {ratio:.6f}"
        else:
            kept.append(col)
    
    return kept, removed_with_reasons


def filter_duplicate_columns(df, columns):
    """Remove _dup columns that are duplicates."""
    kept = []
    removed_with_reasons = {}
    seen_bases = set()
    
    for col in columns:
        dup_match = re.match(r'(.+)_dup\d+$', col)
        if dup_match:
            base = dup_match.group(1)
            if base in seen_bases:
                removed_with_reasons[col] = f"Duplicate of {base}"
            else:
                kept.append(col)
                seen_bases.add(base)
        else:
            kept.append(col)
            seen_bases.add(col)
    
    return kept, removed_with_reasons


def convert_to_numeric(df, columns):
    """Convert selected columns to numeric dtype."""
    df_out = df.copy()
    for col in columns:
        if col in df_out.columns:
            df_out[col] = pd.to_numeric(df_out[col], errors='coerce')
    return df_out


def generate_report(bank_name, original_count, col_types, filter_results, final_columns):
    """Generate a per-bank filtering report."""
    lines = [
        "=" * 70,
        f"FFIEC FEATURE FILTERING REPORT - {bank_name}",
        f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        "=" * 70,
        "",
        "SUMMARY",
        "-" * 40,
        f"Original columns:           {original_count:,}",
        f"Metadata columns (kept):    {len(col_types['metadata']):,}",
        f"Non-numeric (removed):      {len(col_types['non_numeric']):,}",
        f"Numeric columns analyzed:   {len(col_types['numeric']):,}",
        f"Final feature columns:      {len(final_columns):,}",
        "",
        "OUTPUT FORMAT: Transposed (rows=features, columns=quarters)",
        "",
        "FILTERING STEPS",
        "-" * 40,
    ]
    
    for step_name, (kept_count, removed_dict) in filter_results.items():
        lines.append(f"\n{step_name}:")
        lines.append(f"  Kept: {kept_count:,} columns")
        lines.append(f"  Removed: {len(removed_dict):,} columns")
        if removed_dict:
            lines.append("  Sample of removed columns:")
            for i, (col, reason) in enumerate(list(removed_dict.items())[:10]):
                lines.append(f"    - {col}: {reason}")
            if len(removed_dict) > 10:
                lines.append(f"    ... and {len(removed_dict) - 10} more")
    
    lines.extend(["", "FINAL FEATURE COLUMNS", "-" * 40, f"Total: {len(final_columns)} columns", ""])
    
    prefixes = {}
    for col in sorted(final_columns):
        prefix = col[:4] if len(col) >= 4 else col
        if prefix not in prefixes:
            prefixes[prefix] = []
        prefixes[prefix].append(col)
    for prefix in sorted(prefixes.keys()):
        cols = prefixes[prefix]
        lines.append(f"{prefix}*: {len(cols)} columns")
    
    return "\n".join(lines)


# =============================================================================
# PER-BANK FILTERING PIPELINE
# =============================================================================

def filter_single_bank(filepath, bank_name):
    """
    Run the full feature-filtering pipeline on a single bank's CSV.
    Returns (output_df, feature_columns, filter_summary_dict).
    """
    print(f"\n  Loading {filepath.name}...")
    df = pd.read_csv(filepath, low_memory=False)
    print(f"    Loaded: {df.shape[0]:,} rows x {df.shape[1]:,} columns")
    original_count = len(df.columns)
    
    # Categorize columns
    col_types = identify_column_types(df)
    print(f"    Metadata: {len(col_types['metadata']):,} | Numeric: {len(col_types['numeric']):,} | Non-numeric: {len(col_types['non_numeric']):,}")
    
    filter_results = {}
    current_columns = col_types['numeric']
    
    # Filter by null percentage
    kept, removed = filter_by_null_percentage(df, current_columns, NULL_THRESHOLD)
    filter_results['Null Percentage Filter'] = (len(kept), removed)
    print(f"    Null filter:      {len(kept):,} kept, {len(removed):,} removed")
    current_columns = kept
    
    # Filter by variance
    kept, removed = filter_by_variance(df, current_columns, VARIANCE_THRESHOLD)
    filter_results['Variance Filter'] = (len(kept), removed)
    print(f"    Variance filter:  {len(kept):,} kept, {len(removed):,} removed")
    current_columns = kept
    
    # Filter by unique ratio
    kept, removed = filter_by_unique_ratio(df, current_columns, UNIQUE_RATIO_THRESHOLD)
    filter_results['Unique Ratio Filter'] = (len(kept), removed)
    print(f"    Unique filter:    {len(kept):,} kept, {len(removed):,} removed")
    current_columns = kept
    
    # Filter duplicates
    kept, removed = filter_duplicate_columns(df, current_columns)
    filter_results['Duplicate Filter'] = (len(kept), removed)
    print(f"    Duplicate filter: {len(kept):,} kept, {len(removed):,} removed")
    final_feature_columns = kept
    
    # Build output
    output_columns = col_types['metadata'] + final_feature_columns
    output_columns = [c for c in output_columns if c in df.columns]
    df_out = df[output_columns].copy()
    df_out = convert_to_numeric(df_out, final_feature_columns)
    
    print(f"    => {len(final_feature_columns):,} features + {len(col_types['metadata']):,} metadata = {len(output_columns):,} columns")
    
    # Generate report
    report = generate_report(bank_name, original_count, col_types, filter_results, final_feature_columns)
    
    return df_out, final_feature_columns, report


# =============================================================================
# TRANSPOSE: Features as rows, Quarters as columns  (v0.2 addition)
# =============================================================================

def parse_quarter_to_date(quarter_str):
    """
    Convert FFIEC quarter string to a sortable datetime.
    Handles multiple possible formats:
      - 'FFIEC CDR Call Bulk All Schedules 03312024'  (MMDDYYYY at end)
      - '03312024' (just the date portion)
      - '2024-03-31' (ISO format)
      - '03/31/2024' (US date format)
    """
    s = str(quarter_str).strip()
    
    # Try 1: Extract 8-digit date from end of string (MMDDYYYY)
    match = re.search(r'(\d{8})$', s)
    if match:
        ds = match.group(1)
        try:
            return datetime(int(ds[4:8]), int(ds[0:2]), int(ds[2:4]))
        except ValueError:
            pass
    
    # Try 2: Extract 8-digit date from anywhere in string
    match = re.search(r'(\d{8})', s)
    if match:
        ds = match.group(1)
        try:
            return datetime(int(ds[4:8]), int(ds[0:2]), int(ds[2:4]))
        except ValueError:
            pass
    
    # Try 3: Common date formats
    for fmt in ('%Y-%m-%d', '%m/%d/%Y', '%m-%d-%Y', '%Y-%m-%d %H:%M:%S'):
        try:
            return datetime.strptime(s, fmt)
        except (ValueError, TypeError):
            continue
    
    # Try 4: Let pandas try
    try:
        result = pd.to_datetime(s)
        if pd.notna(result):
            return result.to_pydatetime()
    except:
        pass
    
    return None


def transpose_bank_df(df, feature_columns, quarter_column='quarter'):
    """
    Transpose a bank DataFrame so that:
      - Rows    = feature names  (index column named 'feature')
      - Columns = quarter labels (sorted chronologically, formatted MM/DD/YYYY)

    Metadata columns are dropped; only numeric feature data is kept.

    Returns:
      df_transposed: DataFrame with features as rows and quarters as columns
      quarter_labels: list of quarter column headers in chronological order
    """
    df_work = df.copy()

    # Debug: show what quarter values look like
    sample_quarters = df_work[quarter_column].head(3).tolist()
    print(f"    Sample quarter values: {sample_quarters}")

    # Parse quarter strings into dates for sorting
    df_work['_qdate'] = pd.to_datetime(df_work[quarter_column].apply(parse_quarter_to_date))
    
    # Debug: check how many parsed successfully
    n_parsed = df_work['_qdate'].notna().sum()
    n_total = len(df_work)
    print(f"    Quarter parsing: {n_parsed}/{n_total} parsed successfully")
    
    # Drop rows where quarter couldn't be parsed
    if n_parsed < n_total:
        print(f"    WARNING: Dropping {n_total - n_parsed} rows with unparseable quarters")
        df_work = df_work[df_work['_qdate'].notna()].copy()
    
    if len(df_work) == 0:
        raise ValueError("No quarter values could be parsed. Check the 'quarter' column format.")
    
    df_work = df_work.sort_values('_qdate').reset_index(drop=True)

    # Build clean quarter labels: MM/DD/YYYY
    quarter_labels = df_work['_qdate'].dt.strftime('%m/%d/%Y').tolist()
    
    # Verify no NaN labels
    assert all(isinstance(q, str) and q != 'NaT' for q in quarter_labels), \
        f"Quarter label conversion failed. Sample: {quarter_labels[:3]}"

    # Extract only feature columns, transpose
    feature_data = df_work[feature_columns].copy()
    
    # Set the index to quarter labels BEFORE transposing
    # so that after .T, the columns will be the quarter labels
    feature_data.index = quarter_labels
    feature_data.index.name = 'quarter'
    
    # Transpose: rows become columns and vice versa
    df_transposed = feature_data.T
    df_transposed.index.name = 'feature'
    
    # Verify columns are quarter strings, not integers
    assert isinstance(df_transposed.columns[0], str), \
        f"Expected string column names, got {type(df_transposed.columns[0])}: {df_transposed.columns[0]}"

    return df_transposed, quarter_labels


# =============================================================================
# MAIN
# =============================================================================

def main():
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    print("=" * 70)
    print("FFIEC FEATURE FILTERING PIPELINE (Step 3 Track 2 - Per-Bank v0.2)")
    print("  Output format: TRANSPOSED (rows=features, columns=quarters)")
    print("=" * 70)
    
    # Discover per-bank CSVs
    bank_files = sorted(INPUT_DIR.glob("ffiec_*.csv"))
    # Exclude the combined file if present
    bank_files = [f for f in bank_files if 'filtered_banks' not in f.name]
    
    if not bank_files:
        print(f"\nERROR: No per-bank CSV files found in {INPUT_DIR}/")
        print("  Run Step 2 Track 2 first.")
        return None
    
    print(f"\nFound {len(bank_files)} per-bank files in {INPUT_DIR}/")
    
    # Process each bank
    all_results = {}  # bank_name -> {features, n_rows, n_features, filepath}
    
    for filepath in bank_files:
        # Derive bank name from filename
        # e.g., ffiec_bank_of_america.csv -> bank_of_america
        slug = filepath.stem.replace('ffiec_', '')
        
        # Find the matching display name
        bank_name = slug.replace('_', ' ').title()
        for rssd, name in BANKS.items():
            if name.lower().replace(' ', '_').replace('.', '').replace(',', '') == slug:
                bank_name = name
                break
        
        print(f"\n{'─'*70}")
        print(f"  {bank_name}")
        print(f"{'─'*70}")
        
        df_out, feature_cols, report = filter_single_bank(filepath, bank_name)
        
        # ---- v0.2: Transpose before saving ----
        df_transposed, quarter_labels = transpose_bank_df(df_out, feature_cols)
        n_features = len(feature_cols)
        n_quarters = len(quarter_labels)
        print(f"    Transposed: {n_features:,} features x {n_quarters} quarters")
        print(f"    Quarter range: {quarter_labels[0]} -> {quarter_labels[-1]}")
        print(f"    Verify columns: {list(df_transposed.columns[:3])} ... {list(df_transposed.columns[-3:])}")
        
        # Save transposed output
        out_csv = OUTPUT_DIR / f"ffiec_{slug}_features.csv"
        out_report = OUTPUT_DIR / f"filtering_report_{slug}.txt"
        
        df_transposed.to_csv(out_csv)  # index=True is default, writes 'feature' as first col
        with open(out_report, 'w') as f:
            f.write(report)
        
        all_results[bank_name] = {
            'features': set(feature_cols),
            'n_quarters': n_quarters,
            'n_features': n_features,
            'filepath': out_csv.name,
        }
    
    # =========================================================================
    # Cross-bank comparison
    # =========================================================================
    print(f"\n\n{'='*70}")
    print("CROSS-BANK FEATURE COMPARISON")
    print(f"{'='*70}\n")
    
    # Features common to ALL banks
    all_feature_sets = [info['features'] for info in all_results.values()]
    common_features = set.intersection(*all_feature_sets) if all_feature_sets else set()
    union_features = set.union(*all_feature_sets) if all_feature_sets else set()
    
    print(f"Features common to ALL banks: {len(common_features):,}")
    print(f"Features in ANY bank:         {len(union_features):,}")
    print(f"Bank-specific features:       {len(union_features) - len(common_features):,}")
    
    print(f"\nPer-bank summary:")
    for bank_name, info in all_results.items():
        unique_count = len(info['features'] - common_features)
        print(f"  {bank_name:30s} | {info['n_quarters']:>4} quarters | {info['n_features']:>4} features | {unique_count:>3} unique to this bank")
    
    # Save cross-bank comparison report
    comparison_lines = [
        "=" * 70,
        "CROSS-BANK FEATURE COMPARISON",
        f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        "=" * 70,
        "",
        "Output format: TRANSPOSED (rows=features, columns=quarters)",
        "",
        f"Features common to ALL banks: {len(common_features)}",
        f"Features in ANY bank: {len(union_features)}",
        "",
        "PER-BANK SUMMARY",
        "-" * 40,
    ]
    for bank_name, info in all_results.items():
        comparison_lines.append(f"{bank_name}: {info['n_features']} features, {info['n_quarters']} quarters")
    
    comparison_lines.extend(["", "COMMON FEATURES (in all banks)", "-" * 40])
    for col in sorted(common_features):
        comparison_lines.append(f"  {col}")
    
    comparison_lines.extend(["", "BANK-SPECIFIC FEATURES", "-" * 40])
    for bank_name, info in all_results.items():
        unique = sorted(info['features'] - common_features)
        if unique:
            comparison_lines.append(f"\n{bank_name} only ({len(unique)}):")
            for col in unique:
                comparison_lines.append(f"  {col}")
    
    with open(OUTPUT_DIR / "feature_comparison.txt", 'w') as f:
        f.write("\n".join(comparison_lines))
    
    # Final summary
    print(f"\n{'='*70}")
    print("COMPLETE")
    print(f"{'='*70}")
    print(f"\nOutputs saved to {OUTPUT_DIR.resolve()}/")
    print(f"  Per-bank feature CSVs:  {len(all_results)}  (transposed: rows=features, cols=quarters)")
    print(f"  Per-bank reports:       {len(all_results)}")
    print(f"  Cross-bank comparison:  feature_comparison.txt")
    print(f"{'='*70}\n")
    
    return all_results


if __name__ == "__main__":
    results = main()

FFIEC FEATURE FILTERING PIPELINE (Step 3 Track 2 - Per-Bank v0.2)
  Output format: TRANSPOSED (rows=features, columns=quarters)

Found 6 per-bank files in per_bank/

──────────────────────────────────────────────────────────────────────
  Bank of America
──────────────────────────────────────────────────────────────────────

  Loading ffiec_bank_of_america.csv...
    Loaded: 99 rows x 6,444 columns
    Metadata: 11 | Numeric: 5,731 | Non-numeric: 702
    Null filter:      462 kept, 5,269 removed
    Variance filter:  433 kept, 29 removed
    Unique filter:    430 kept, 3 removed
    Duplicate filter: 430 kept, 0 removed
    => 430 features + 11 metadata = 441 columns
    Sample quarter values: ['03/31/2001', '03/31/2002', '03/31/2003']
    Quarter parsing: 99/99 parsed successfully
    Transposed: 430 features x 99 quarters
    Quarter range: 03/31/2001 -> 09/30/2025
    Verify columns: ['03/31/2001', '06/30/2001', '09/30/2001'] ... ['03/31/2025', '06/30/2025', '09/30/2025']

─────────